# Downloading Multisource Remote Sensing Data Using Python

This notebook demonstrates how to download and process various remote sensing datasets, including Sentinel-1, Sentinel-2, MODIS, DEM, and CHIRPS data. The focus is on simplifying the workflow for accessing these datasets using Python. 

### Prerequisites
- Python 3.7 or later
- Required Libraries: `rasterio`, `geopandas`, `asf_search`, `modis_tools`, etc.
- Internet access for downloading datasets

### Structure
1. **Sentinel-2 Data**
2. **Digital Elevation Model (DEM)**
3. **Precipitation Data (CHIRPS)**
4. **MODIS Data**
5. **Sentinel-1 Data**

Each section includes example code for querying, downloading, and processing data. Comments and references are provided to guide users through the workflow.

## Taking a look at available Sen2 dataset's metadata

In [ ]:
import geopandas as gpd
from pystac_client import Client

# Load the AOI from a shapefile
shapefile_path = "path/shapefile/combined_extent.shp"  # Replace with the path to your shapefile
gdf = gpd.read_file(shapefile_path)
geometry = gdf.geometry.iloc[0].__geo_interface__  # Assuming the AOI is the first geometry in the shapefile

# Use publicly available STAC link
client = Client.open("https://earth-search.aws.element84.com/v1")

# ID of the collection
collection = "sentinel-2-l2a"

# Date range
date_range = "2022-09-01/2022-10-15"

# Run pystac client search without filters to check available cloud cover
search = client.search(
    collections=[collection], intersects=geometry, datetime=date_range
)

# Check metadata to understand cloud cover properties
print("Available scenes and their metadata:")
for item in search.items():
    print(f"Scene ID: {item.id}")
    for key, value in item.properties.items():
        print(f"{key}: {value}")
    print("-" * 40)


## Downloading dataset with specified filters

In [ ]:
import os
import requests
import zipfile
import geopandas as gpd
from pystac_client import Client
### This is publically available dataset at AWS dont need any credentianls
# Load the AOI from a shapefile
shapefile_path = "path/shapefile/combined_extent.shp"  # Replace with the path to your shapefile
gdf = gpd.read_file(shapefile_path)
geometry = gdf.geometry.iloc[0].__geo_interface__  # Assuming the AOI is the first geometry in the shapefile

# Use publicly available STAC link
client = Client.open("https://earth-search.aws.element84.com/v1")

# ID of the collection
collection = "sentinel-2-l2a"

# Date range
date_range = "2022-09-01/2022-11-15"

# Define filters (e.g., cloud cover less than 20%)
filters = {
    "eo:cloud_cover": {"lt": 25}  # Using an integer for the cloud cover filter
}

# Run pystac client search with the filters
search = client.search(
    collections=[collection],
    intersects=geometry,
    datetime=date_range,
    query=filters
)m

# Get the total number of scenes found
total_scenes = len(list(search.items()))
print(f"Total scenes found with the specified criteria: {total_scenes}")

# Specify the local directory where you want to save the data
local_dir = "path/download/Sen2"
os.makedirs(local_dir, exist_ok=True)

# Extract, download assets, and organize them per scene
for item in search.items():
    try:
        # Create a directory for the scene using the item ID
        scene_dir = os.path.join(local_dir, item.id)
        os.makedirs(scene_dir, exist_ok=True)
        
        # Save metadata information to a text file
        metadata_filename = os.path.join(scene_dir, f"{item.id}_metadata.txt")
        with open(metadata_filename, 'w') as meta_file:
            meta_file.write(f"Scene ID: {item.id}\n")
            meta_file.write(f"Date and Time: {item.datetime}\n")
            meta_file.write(f"Cloud Cover: {item.properties.get('eo:cloud_cover', 'N/A')}\n")
            meta_file.write(f"Other Metadata: {item.properties}\n")

        # Download and save assets
        for asset_key, asset in item.assets.items():
            # Skip unnecessary files
            if asset_key in ['granule_metadata', 'thumbnail', 'tileinfo_metadata']:
                continue
            
            # Get the URL of the asset
            url = asset.href
            # Create a local filename with the scene ID as prefix
            local_filename = os.path.join(scene_dir, f"{item.id}_{asset_key}.tif")
            
            # Download the asset
            print(f"Downloading {url} to {local_filename}...")
            response = requests.get(url, stream=True)
            if response.status_code == 200:
                with open(local_filename, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                print(f"Downloaded {local_filename}")
            else:
                print(f"Failed to download {url} with status code {response.status_code}")

        # Optionally zip the scene directory without deleting the original files
        zip_filename = os.path.join(local_dir, f"{item.id}.zip")
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for root, _, files in os.walk(scene_dir):
                for file in files:
                    zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), scene_dir))
        
        print(f"Scene {item.id} zipped as {zip_filename}")

    except Exception as e:
        print(f"An error occurred while processing scene {item.id}: {e}")

print("Download and zipping complete.")


## Downloading Sen2 data for multiple shapefiles

In [ ]:
import os
import requests
import zipfile
import geopandas as gpd
from pystac_client import Client

# Path to the directory containing all shapefiles
shapefile_dir = "path/Sen2/AOI"  # Replace with the path to your shapefile directory
local_dir_base = "path/Sen2/download/Sen2"  # Base directory to save downloaded data

# Public STAC link
client = Client.open("https://earth-search.aws.element84.com/v1")

# ID of the collection
collection = "sentinel-2-l2a"

# Date range
date_range = "2022-09-01/2022-10-15"

# Define filters (e.g., cloud cover less than 25%)
filters = {
    "eo:cloud_cover": {"lt": 25}
}

# Loop through all shapefiles in the directory
for shapefile_name in os.listdir(shapefile_dir):
    if shapefile_name.endswith(".shp"):
        shapefile_path = os.path.join(shapefile_dir, shapefile_name)
        shapefile_base_name = os.path.splitext(shapefile_name)[0]  # Get shapefile name without extension
        
        # Load the AOI from the shapefile
        gdf = gpd.read_file(shapefile_path)
        geometry = gdf.geometry.iloc[0].__geo_interface__  # Assuming the AOI is the first geometry

        # Run pystac client search with the filters
        search = client.search(
            collections=[collection],
            intersects=geometry,
            datetime=date_range,
            query=filters
        )

        # Get the total number of scenes found
        total_scenes = len(list(search.items()))
        print(f"Total scenes found for {shapefile_name} with the specified criteria: {total_scenes}")

        # Create a directory for each shapefile
        local_dir = os.path.join(local_dir_base, shapefile_base_name)
        os.makedirs(local_dir, exist_ok=True)

        # Extract, download assets, and organize them per scene
        for item in search.items():
            try:
                # Create a directory for the scene using the item ID
                scene_dir = os.path.join(local_dir, item.id)
                os.makedirs(scene_dir, exist_ok=True)

                # Save metadata information to a text file
                metadata_filename = os.path.join(scene_dir, f"{item.id}_metadata.txt")
                with open(metadata_filename, 'w') as meta_file:
                    meta_file.write(f"Scene ID: {item.id}\n")
                    meta_file.write(f"Date and Time: {item.datetime}\n")
                    meta_file.write(f"Cloud Cover: {item.properties.get('eo:cloud_cover', 'N/A')}\n")
                    meta_file.write(f"Other Metadata: {item.properties}\n")

                # Download and save assets
                for asset_key, asset in item.assets.items():
                    # Skip unnecessary files
                    if asset_key in ['granule_metadata', 'thumbnail', 'tileinfo_metadata']:
                        continue
                    
                    # Get the URL of the asset
                    url = asset.href
                    # Create a local filename with the scene ID as prefix
                    local_filename = os.path.join(scene_dir, f"{item.id}_{asset_key}.tif")
                    
                    # Download the asset
                    print(f"Downloading {url} to {local_filename}...")
                    response = requests.get(url, stream=True)
                    if response.status_code == 200:
                        with open(local_filename, 'wb') as f:
                            for chunk in response.iter_content(chunk_size=8192):
                                f.write(chunk)
                        print(f"Downloaded {local_filename}")
                    else:
                        print(f"Failed to download {url} with status code {response.status_code}")

                # Optionally zip the scene directory without deleting the original files
                zip_filename = os.path.join(local_dir, f"{item.id}.zip")
                with zipfile.ZipFile(zip_filename, 'w') as zipf:
                    for root, _, files in os.walk(scene_dir):
                        for file in files:
                            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), scene_dir))

                print(f"Scene {item.id} zipped as {zip_filename}")

            except Exception as e:
                print(f"An error occurred while processing scene {item.id}: {e}")

print("Download and zipping complete for all shapefiles.")


## Checking overlap with shapefile before download and using multiprocessing

In [ ]:
import os
import requests
import zipfile
import geopandas as gpd
from pystac_client import Client
from shapely.geometry import shape
from joblib import Parallel, delayed

# Path to the directory containing all shapefiles
shapefile_dir = "path/Sen2/AOI"  # Replace with the path to your shapefile directory
local_dir_base = "path/Sen2/download/Sen2_test"  # Base directory to save downloaded data

# Public STAC link
client = Client.open("https://earth-search.aws.element84.com/v1")

# ID of the collection
collection = "sentinel-2-l2a"

# Date range
date_range = "2022-09-01/2022-10-15"

# Define filters (e.g., cloud cover less than 25%)
filters = {
    "eo:cloud_cover": {"lt": 45}
}

# Function to check if the scene geometry contains the shapefile geometry (100% overlap)
def is_full_overlap(scene_geometry, aoi_geometry):
    scene_shape = shape(scene_geometry)
    return scene_shape.contains(aoi_geometry)  # Check if the scene contains the AOI

# Function to perform pre-check: ensure all shapefiles have 100% overlap
def precheck_shapefile_overlap(shapefile_name):
    shapefile_path = os.path.join(shapefile_dir, shapefile_name)
    shapefile_base_name = os.path.splitext(shapefile_name)[0]  # Get shapefile name without extension

    # Load the AOI from the shapefile
    gdf = gpd.read_file(shapefile_path)
    aoi_geometry = gdf.geometry.iloc[0]  # Assuming the AOI is the first geometry

    # Run pystac client search with the filters
    search = client.search(
        collections=[collection],
        intersects=aoi_geometry.__geo_interface__,  # Query based on the AOI geometry
        datetime=date_range,
        query=filters
    )

    # Inform user how many scenes found before filtering for 100% overlap
    all_scenes = list(search.items())
    print(f"Total scenes found for {shapefile_name} before 100% overlap check: {len(all_scenes)}")

    # Filter for scenes that fully contain the AOI
    valid_scenes = []
    for item in all_scenes:
        scene_geometry = item.geometry
        if is_full_overlap(scene_geometry, aoi_geometry):
            valid_scenes.append(item)

    # Return True if at least one valid scene is found, else False
    if len(valid_scenes) == 0:
        print(f"No fully overlapping scenes found for {shapefile_name}.")
        return False
    else:
        print(f"Total scenes with 100% overlap for {shapefile_name}: {len(valid_scenes)}")
        return True

# Function to process downloading and zipping for a single scene
def process_scene(item, scene_dir, local_dir):
    try:
        # Create a directory for the scene using the item ID
        os.makedirs(scene_dir, exist_ok=True)

        # Save metadata information to a text file
        metadata_filename = os.path.join(scene_dir, f"{item.id}_metadata.txt")
        with open(metadata_filename, 'w') as meta_file:
            meta_file.write(f"Scene ID: {item.id}\n")
            meta_file.write(f"Date and Time: {item.datetime}\n")
            meta_file.write(f"Cloud Cover: {item.properties.get('eo:cloud_cover', 'N/A')}\n")
            meta_file.write(f"Other Metadata: {item.properties}\n")

        # Download and save assets
        for asset_key, asset in item.assets.items():
            # Skip unnecessary files
            if asset_key in ['granule_metadata', 'thumbnail', 'tileinfo_metadata']:
                continue

            # Get the URL of the asset
            url = asset.href
            # Create a local filename with the scene ID as prefix
            local_filename = os.path.join(scene_dir, f"{item.id}_{asset_key}.tif")

            # Download the asset
            print(f"Downloading {url} to {local_filename}...")
            response = requests.get(url, stream=True)
            if response.status_code == 200:
                with open(local_filename, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                print(f"Downloaded {local_filename}")
            else:
                print(f"Failed to download {url} with status code {response.status_code}")

        # Optionally zip the scene directory without deleting the original files
        zip_filename = os.path.join(local_dir, f"{item.id}.zip")
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for root, _, files in os.walk(scene_dir):
                for file in files:
                    zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), scene_dir))

        print(f"Scene {item.id} zipped as {zip_filename}")

    except Exception as e:
        print(f"An error occurred while processing scene {item.id}: {e}")

# Function to handle all scenes for a single shapefile
def process_shapefile(shapefile_name):
    shapefile_path = os.path.join(shapefile_dir, shapefile_name)
    shapefile_base_name = os.path.splitext(shapefile_name)[0]  # Get shapefile name without extension

    # Load the AOI from the shapefile
    gdf = gpd.read_file(shapefile_path)
    aoi_geometry = gdf.geometry.iloc[0]  # Assuming the AOI is the first geometry

    # Run pystac client search with the filters
    search = client.search(
        collections=[collection],
        intersects=aoi_geometry.__geo_interface__,  # Query based on the AOI geometry
        datetime=date_range,
        query=filters
    )

    # Filter for scenes that fully contain the AOI
    valid_scenes = []
    for item in search.items():
        scene_geometry = item.geometry
        if is_full_overlap(scene_geometry, aoi_geometry):
            valid_scenes.append(item)

    total_scenes = len(valid_scenes)

    print(f"Total scenes with 100% overlap for {shapefile_name}: {total_scenes}")

    # Create a directory for each shapefile
    local_dir = os.path.join(local_dir_base, shapefile_base_name)
    os.makedirs(local_dir, exist_ok=True)

    # Process each valid scene in parallel
    Parallel(n_jobs=3)(delayed(process_scene)(item, os.path.join(local_dir, item.id), local_dir) for item in valid_scenes)


# Main logic: Pre-check if all shapefiles have at least one 100% overlap scene before proceeding with downloads
if __name__ == "__main__":
    shapefiles = [f for f in os.listdir(shapefile_dir) if f.endswith(".shp")]
    
    # Step 1: Pre-check for 100% overlap for all shapefiles
    all_overlap = True
    for shapefile_name in shapefiles:
        if not precheck_shapefile_overlap(shapefile_name):
            all_overlap = False
    
    if not all_overlap:
        print("Some shapefiles do not have 100% overlap scenes. Halting the process.")
    else:
        # Step 2: Proceed with downloading if all shapefiles have 100% overlap scenes
        print("All shapefiles have 100% overlap. Proceeding with downloads.")
        Parallel(n_jobs=3)(delayed(process_shapefile)(shapefile_name) for shapefile_name in shapefiles)

    print("Process completed.")


## Downloading Corresponding DEMs

In [ ]:
## No login credentials required
import os
import numpy as np
from numpy import floor
import geopandas as gpd
from subprocess import call

# Define the AOI and destination CRS
shapefile_path = "ex/shapefile/combined_extent.shp"  # Replace with your shapefile path
destination_crs = "EPSG:4326"  # Can be None if CRS conversion is not needed

# Define paths
base_folder = "ex/download/DEM"
download_path = f'{base_folder}/dems/copernicus'
crs_conversion_path = f'{base_folder}/dems/crs_conversion'

# Create the DEM folder if necessary
if not os.path.exists(download_path):
    print(f"Creating Folder(s): {download_path}")
    os.makedirs(download_path)

# Load the AOI from a shapefile
aoi = gpd.read_file(shapefile_path)
aoi = aoi.to_crs('EPSG:4326')

# Function to construct the full path to the DEM on AWS S3
def s3Path(lat, lon):
    """ 
    Construct full path to data, using longitude and latitude.
    """
    lonSign = {1: 'E', -1: 'W', 0: 'E'}
    latSign = {1: 'N', -1: 'S', 0: 'N'}

    lonStr = f'{lonSign[np.sign(lon)]}{np.abs(lon):03}' # The sign function returns -1 if x < 0, 0 if x==0, 1 if x > 0
    latStr =  f'{latSign[np.sign(lat)]}{np.abs(lat):02}'

    myPath = f's3://copernicus-dem-30m/Copernicus_DSM_COG_10_{latStr}_00_{lonStr}_00_DEM/Copernicus_DSM_COG_10_{latStr}_00_{lonStr}_00_DEM.tif'
    
    return myPath

# Download DEM data for each point in the AOI

# For each record in the AOI gdf
for row in aoi.itertuples():
    # Get the geometry bounds for that record.
    geom = row.geometry
    xmin, ymin, xmax, ymax = geom.bounds

    # Add a buffer to the bounds to ensure we cover the area
    buffer_degree = 0.05
    xmin -= buffer_degree
    ymin -= buffer_degree
    xmax += buffer_degree
    ymax += buffer_degree

    # Get a list of every whole-degree longitude and latitude in the buffered bounds.
    lons = list(set([int(floor(xmin)), int(floor(xmax))]))
    lats = list(set([int(floor(ymin)), int(floor(ymax))]))

    # For each lon/lat combination, call the AWS CLI to download the corresponding DEM data.
    for lon in lons:
        for lat in lats:
            myPath = s3Path(lat, lon)
            myFile = myPath.split('/')[-1]

            if not os.path.exists(f'{download_path}/{myFile}'):
                print(f"Downloading: {myPath}")
                command = f'aws s3 cp {myPath} {download_path} --no-sign-request'
                print(command)
                call(command, shell=True)
            else:
                print(f'Already exists: {myFile}')

# Optionally convert CRS if a destination CRS is specified
if destination_crs is not None:
    # convert_tif_folder_to_crs(download_path, crs_conversion_path, destination_crs)  # Commented out for now
    print(f"Skipping CRS conversion; function not implemented.")

print('END of Downloads')


## Downloading DEM for mutiple shapefile

In [ ]:
import os
import numpy as np
from numpy import floor
import geopandas as gpd
from subprocess import call

# Define the base paths
shapefile_dir = "ex/Sen2/shapefile/"  # Folder containing all shapefiles
destination_crs = "EPSG:4326"  # Can be None if CRS conversion is not needed

# Define base paths for downloads
base_folder = "ex/Sen2/download/DEM"
download_base_path = f'{base_folder}/dems'

# Function to construct the full path to the DEM on AWS S3
def s3Path(lat, lon):
    """ 
    Construct full path to data, using longitude and latitude.
    """
    lonSign = {1: 'E', -1: 'W', 0: 'E'}
    latSign = {1: 'N', -1: 'S', 0: 'N'}

    lonStr = f'{lonSign[np.sign(lon)]}{np.abs(lon):03}'  # The sign function returns -1 if x < 0, 0 if x==0, 1 if x > 0
    latStr =  f'{latSign[np.sign(lat)]}{np.abs(lat):02}'

    myPath = f's3://copernicus-dem-30m/Copernicus_DSM_COG_10_{latStr}_00_{lonStr}_00_DEM/Copernicus_DSM_COG_10_{latStr}_00_{lonStr}_00_DEM.tif'
    
    return myPath

# Function to process each shapefile
def process_shapefile(shapefile_path):
    shapefile_name = os.path.splitext(os.path.basename(shapefile_path))[0]  # Extract shapefile name without extension
    download_path = f'{download_base_path}/{shapefile_name}'
    
    # Create the download folder if necessary
    if not os.path.exists(download_path):
        print(f"Creating Folder(s): {download_path}")
        os.makedirs(download_path)

    # Load the AOI from a shapefile
    aoi = gpd.read_file(shapefile_path)
    aoi = aoi.to_crs(destination_crs)

    # For each record in the AOI gdf
    for row in aoi.itertuples():
        # Get the geometry bounds for that record.
        geom = row.geometry
        xmin, ymin, xmax, ymax = geom.bounds

        # Add a buffer to the bounds to ensure we cover the area
        buffer_degree = 0.05
        xmin -= buffer_degree
        ymin -= buffer_degree
        xmax += buffer_degree
        ymax += buffer_degree

        # Get a list of every whole-degree longitude and latitude in the buffered bounds.
        lons = list(set([int(floor(xmin)), int(floor(xmax))]))
        lats = list(set([int(floor(ymin)), int(floor(ymax))]))

        # For each lon/lat combination, call the AWS CLI to download the corresponding DEM data.
        for lon in lons:
            for lat in lats:
                myPath = s3Path(lat, lon)
                myFile = myPath.split('/')[-1]

                if not os.path.exists(f'{download_path}/{myFile}'):
                    print(f"Downloading: {myPath}")
                    command = f'aws s3 cp {myPath} {download_path} --no-sign-request'
                    print(command)
                    call(command, shell=True)
                else:
                    print(f'Already exists: {myFile}')
                    
    print(f"Finished downloading DEMs for {shapefile_name}")

# Get all shapefiles in the directory
shapefiles = [os.path.join(shapefile_dir, f) for f in os.listdir(shapefile_dir) if f.endswith('.shp')]

# Process each shapefile
for shapefile_path in shapefiles:
    process_shapefile(shapefile_path)

print('END of Downloads')


## Clipping DEM

In [ ]:
## optional, this code clip DEMs using shapefile

import os
import rasterio
import fiona
import geopandas as gpd
from shapely.geometry import box
from rasterio.mask import mask
from pathlib import Path

# Function to check and reproject shapefile if CRS doesn't match with raster
def check_and_reproject_raster(tif_file, shapefile):
    # Open the shapefile
    shp = gpd.read_file(shapefile)
    
    # Open the raster file
    with rasterio.open(tif_file) as src:
        raster_crs = src.crs

    # Get the CRS of the shapefile
    shapefile_crs = shp.crs

    # Check if CRS match
    if raster_crs != shapefile_crs:
        print(f"CRS Mismatch: Raster CRS is {raster_crs}, Shapefile CRS is {shapefile_crs}")
        # Reproject shapefile to match raster CRS
        shp = shp.to_crs(raster_crs)
        print("Shapefile reprojected to match raster CRS.")
        return shp
    else:
        print("CRS match.")
        return shp

# Function to clip shapefile to raster bounds
def clip_shapefile_to_raster(tif_file, shapefile):
    # Open the raster to get its bounds
    with rasterio.open(tif_file) as src:
        raster_bounds = src.bounds
        # Use shapely.geometry.box() to create a bounding box
        raster_bbox = gpd.GeoDataFrame({"geometry": [box(*raster_bounds)]}, crs=src.crs)

    # Open the shapefile and clip it
    shp = gpd.read_file(shapefile)

    # If the shapefile CRS doesn't match, reproject it
    if shp.crs != raster_bbox.crs:
        shp = shp.to_crs(raster_bbox.crs)

    # Clip the shapefile using the raster bounds
    clipped_shapefile = gpd.overlay(shp, raster_bbox, how='intersection')

    return clipped_shapefile

# Function to recursively find all .tif files ending with 'DEM.tif'
def find_tif_files(root_dir):
    tif_files = []
    valid_suffixes = ['DEM.tif']
    
    for dirpath, _, filenames in os.walk(root_dir):
        for file in filenames:
            if any(file.endswith(suffix) for suffix in valid_suffixes):
                tif_files.append(os.path.join(dirpath, file))
    return tif_files

# Function to clip a raster with a shapefile and save it with the shapefile name
def clip_raster_with_shapefile(tif_file, shapefile, output_dir):
    # Check and reproject shapefile to match raster CRS
    shp = check_and_reproject_raster(tif_file, shapefile)
    
    # Clip the shapefile to the raster bounds
    shp = clip_shapefile_to_raster(tif_file, shapefile)

    with rasterio.open(tif_file) as src:
        shapes = [geom for geom in shp.geometry]
        
        if len(shapes) == 0:
            print(f"No overlap between {tif_file} and the shapefile.")
            return
        
        out_image, out_transform = mask(src, shapes, crop=True)
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

        # Create output directory if it doesn't exist
        Path(output_dir).mkdir(parents=True, exist_ok=True)

        # Extract the shapefile name (without extension) to use it as the output filename
        shapefile_name = os.path.splitext(os.path.basename(shapefile))[0]
        output_file = os.path.join(output_dir, f"{shapefile_name}_DEM.tif")
        
        # Save the clipped raster
        with rasterio.open(output_file, "w", **out_meta) as dest:
            dest.write(out_image)

# Function to match root folder names with shapefiles and clip rasters in subfolders
def clip_all_tifs_with_matching_shapefile(tif_root_dir, shapefile_root_dir):
    # Iterate over the subfolders in the tif_root_dir
    for root_folder in os.listdir(tif_root_dir):
        root_folder_path = os.path.join(tif_root_dir, root_folder)
        
        # Check if the root folder is indeed a directory
        if os.path.isdir(root_folder_path):
            # Try to find a shapefile in the shapefile root directory with the same name as the root folder
            shapefile_path = os.path.join(shapefile_root_dir, f"{root_folder}.shp")
            
            if os.path.exists(shapefile_path):
                print(f"Using shapefile: {shapefile_path} for folder: {root_folder}")
                
                # Find all .tif files within this root folder (including subfolders)
                tif_files = find_tif_files(root_folder_path)
                
                # Process each .tif file
                for tif_file in tif_files:
                    # Recreate the folder structure in the output directory
                    relative_path = os.path.relpath(tif_file, tif_root_dir)
                    output_dir = os.path.join(tif_root_dir, "clipped", os.path.dirname(relative_path))
                    
                    # Clip each raster using the matched shapefile
                    clip_raster_with_shapefile(tif_file, shapefile_path, output_dir)
                    print(f"Clipped: {tif_file} and saved to {output_dir}")
            else:
                print(f"No matching shapefile found for folder: {root_folder}")

if __name__ == "__main__":
    # Define the root directory containing the .tif files and the shapefile root directory
    tif_root_directory = r"path\download\DEM\dems"  # Your specified root directory for .tif files
    shapefile_root_directory = r"path\AOI"  # Your specified root directory for shapefiles

    # Call the function to clip all .tif files with matching shapefiles
    clip_all_tifs_with_matching_shapefile(tif_root_directory, shapefile_root_directory)


## Dwonloading Precipitation (CHIRPS) dataset

In [ ]:
# No login credential required, download CIRPS precipitation data
import os
import requests
import gzip
import shutil
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import pandas as pd

# Define paths
base_folder = "path/download/CHIRPS"
download_path = f'{base_folder}/daily'
os.makedirs(download_path, exist_ok=True)

# Load the AOI from a shapefile and reproject to WGS 84 (EPSG:4326)
shapefile_path = "path/shapefile/combined_extent.shp"
gdf = gpd.read_file(shapefile_path)
gdf = gdf.to_crs('EPSG:4326')  # Reproject to the CRS used by CHIRPS

geometry = gdf.geometry

# Function to download CHIRPS data via HTTP
def download_chirps_http(date):
    year = date[:4]
    month = date[5:7]
    day = date[8:10]
    filename = f'chirps-v2.0.{year}.{month}.{day}.tif.gz'
    url = f'http://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/tifs/p05/{year}/{filename}'
    local_gz_path = os.path.join(download_path, filename)

    if not os.path.exists(local_gz_path):
        print(f'Downloading: {url}')
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(local_gz_path, 'wb') as f_out:
                f_out.write(response.content)
            print(f'Downloaded {local_gz_path}')
        else:
            print(f'Failed to download {url} with status code {response.status_code}')
    else:
        print(f'Already exists: {local_gz_path}')

# Define date range (example: 2022-01-01 to 2022-01-31)
start_date = '2022-01-01'
end_date = '2022-01-10'
date_range = pd.date_range(start=start_date, end=end_date)

# Loop through the date range and download data for each date
for date in date_range:
    download_chirps_http(date.strftime('%Y-%m-%d'))

# Unzip and clip downloaded files
for gz_file in os.listdir(download_path):
    if gz_file.endswith('.gz'):
        gz_path = os.path.join(download_path, gz_file)
        tif_path = gz_path[:-3]  # Remove .gz extension

        # Unzipping
        if not os.path.exists(tif_path):
            print(f'Unzipping {gz_path}')
            with gzip.open(gz_path, 'rb') as f_in:
                with open(tif_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
        
        # Clip to AOI using the exact shape
        with rasterio.open(tif_path) as src:
            out_image, out_transform = mask(src, geometry, crop=True, filled=True)
            out_meta = src.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })
        
            #clipped_path = os.path.join(download_path, f'clipped_{os.path.basename(tif_path)}')
            #with rasterio.open(clipped_path, "w", **out_meta) as dest:
                #dest.write(out_image)
            #print(f'Clipped data saved to {clipped_path}')


## Clipping chirps data

In [ ]:
##optical, this code clip CHIRPS dataset using shapefile

import os
import rasterio
from rasterio.mask import mask
import geopandas as gpd

def clip_raster_with_shapefiles(tif_folder, shapefile_folder, output_folder):
    # List all .tif files in the raster folder
    tif_files = [f for f in os.listdir(tif_folder) if f.endswith('.tif')]
    
    # Ensure output directory exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # List all shapefiles in the folder
    shapefiles = [f for f in os.listdir(shapefile_folder) if f.endswith('.shp')]
    
    # Loop through each .tif file
    for tif_file in tif_files:
        tif_path = os.path.join(tif_folder, tif_file)
        
        # Extract the full date from the .tif file name (assumes 'chirps-v2.0.2022.01.01.tif' format)
        tif_filename = os.path.basename(tif_path)
        # Adjusting the index to get the full date (3rd part from the split)
        date_str = '.'.join(tif_filename.split('.')[2:5])  # Extracts '2022.01.01'
        
        # Open the raster file
        with rasterio.open(tif_path) as src:
            # Loop through each shapefile
            for shapefile in shapefiles:
                shapefile_path = os.path.join(shapefile_folder, shapefile)
                # Read the shapefile using Geopandas
                shapes = gpd.read_file(shapefile_path)
                
                # Reproject shapefile to the same CRS as the raster
                shapes = shapes.to_crs(src.crs)

                # Convert the shapes to GeoJSON-like format for rasterio
                geoms = shapes.geometry.values
                geoms = [geom.__geo_interface__ for geom in geoms]

                # Clip the raster using the geometry
                out_image, out_transform = mask(src, geoms, crop=True)

                # Update metadata
                out_meta = src.meta.copy()
                out_meta.update({
                    "driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform
                })

                # Create the output filename based on the shapefile name and date
                shapefile_name = os.path.splitext(shapefile)[0]
                output_tif_name = f"{shapefile_name}_{date_str}.tif"
                output_tif_path = os.path.join(output_folder, output_tif_name)

                # Save the clipped raster
                with rasterio.open(output_tif_path, 'w', **out_meta) as dest:
                    dest.write(out_image)

                print(f"Saved clipped raster: {output_tif_path}")

# Example usage:
tif_folder = 'path/CHIRPS/daily'
shapefile_folder = r'path/AOI'
output_folder = 'path/download/processed/precipitation'

clip_raster_with_shapefiles(tif_folder, shapefile_folder, output_folder)


## Downloading MODIS data 

In [ ]:
### This code make sure of download band1 and band2 (MOD09GQ) at 250m and band7 at 500m (MOD09GA). Run as per requirement.

from modis_tools.auth import ModisSession
from modis_tools.resources import CollectionApi, GranuleApi
from modis_tools.granule_handler import GranuleHandler
import os
import rasterio
from rasterio.enums import Resampling
import geopandas as gpd

# Set your Earthdata username and password
username = "your_username"  # Update this line
password = "your_password"  # Update this line

# Initialize the MODIS session
session = ModisSession(username=username, password=password)

# Define the local directory where data will be downloaded
download_directory = "path/to/data/download/MODIS/"
os.makedirs(download_directory, exist_ok=True)  # Create the directory if it doesn't exist

# Load the shapefile for the AOI
shapefile_path = "path/to/data/shapefile/combined_extent.shp"  # Replace with your shapefile path
aoi = gpd.read_file(shapefile_path)
aoi = aoi.to_crs('EPSG:4326')  # Reproject to WGS84 if needed
bbox = tuple(aoi.total_bounds)  # Convert numpy array to tuple

# Initialize CollectionApi for 250m and 500m products
collection_client = CollectionApi(session=session)

# Query for MOD09GQ (250m resolution, bands 1 and 2)
collections_250m = collection_client.query(short_name="MOD09GQ", version="061")

# Query for MOD09GA (500m resolution, bands 3-7)
collections_500m = collection_client.query(short_name="MOD09GA", version="061")

# Check if collections are found
if not collections_250m or not collections_500m:
    print("No collections found for MOD09GQ or MOD09GA. Please check the product name and version.")
    exit()

# Function to download granules
def download_granules(collection, start_date, end_date, bounding_box, session, download_path):
    granule_client = GranuleApi.from_collection(collection, session=session)
    granules = granule_client.query(start_date=start_date, end_date=end_date, bounding_box=bounding_box)
    GranuleHandler.download_from_granules(granules, session, path=download_path)
    return granules

# Download 250m data (bands 1 and 2)
granules_250m = download_granules(collections_250m[0], "2022-01-01", "2022-01-03", bbox, session, download_directory)

# Download 500m data (bands 3-7)
granules_500m = download_granules(collections_500m[0], "2022-01-01", "2022-01-03", bbox, session, download_directory)

print(f"Granules downloaded to {download_directory}")

# Function to extract specific bands from the .hdf files and save them in the scene directory
def extract_bands_from_hdf(hdf_path, bands_to_extract, download_dir):
    with rasterio.open(hdf_path) as hdf:
        # Get all available subdatasets
        subdatasets = hdf.subdatasets

        for subdataset_name in subdatasets:
            # Extract the name of the subdataset (e.g., 'MODIS_Grid_500m_2D:sur_refl_b01')
            subdataset_band_name = subdataset_name.split(":")[-1]

            if subdataset_band_name in bands_to_extract:
                with rasterio.open(subdataset_name) as subdataset:
                    # Define scene output directory
                    scene_name = os.path.basename(hdf_path).replace('.hdf', '')
                    scene_output_dir = os.path.join(download_dir, scene_name)
                    os.makedirs(scene_output_dir, exist_ok=True)  # Create directory for each scene

                    # Define output file path for the specific band
                    output_filename = f"{scene_name}_{subdataset_band_name}.tif"
                    full_output_path = os.path.join(scene_output_dir, output_filename)

                    # Read the band data
                    data = subdataset.read(
                        out_shape=(
                            subdataset.count,
                            int(subdataset.height),
                            int(subdataset.width)
                        ),
                        resampling=Resampling.nearest
                    )
                    
                    # Write the data to a new GeoTIFF
                    with rasterio.open(
                        full_output_path,
                        'w',
                        driver='GTiff',
                        height=subdataset.height,
                        width=subdataset.width,
                        count=subdataset.count,
                        dtype=data.dtype,
                        crs=subdataset.crs,
                        transform=subdataset.transform,
                    ) as dst:
                        dst.write(data)
                    
                    print(f"Extracted {subdataset_band_name} to {full_output_path}")

# List of subdataset names corresponding to bands 1 and 2 in MOD09GQ
bands_250m = ["sur_refl_b01", "sur_refl_b02"]

# List of subdataset names corresponding to bands 1, 2, and 7 in MOD09GA (500m resampling)
bands_500m = ["sur_refl_b01", "sur_refl_b02", "sur_refl_b07"]

# Extract bands from each downloaded 250m .hdf file and save to scene-specific folders
for granule in granules_250m:
    hdf_path = os.path.join(download_directory, os.path.basename(granule))
    extract_bands_from_hdf(hdf_path, bands_250m, download_directory)

# Extract bands from each downloaded 500m .hdf file and save to scene-specific folders
for granule in granules_500m:
    hdf_path = os.path.join(download_directory, os.path.basename(granule))
    extract_bands_from_hdf(hdf_path, bands_500m, download_directory)


## Extracting relevant bands

In [ ]:
### This code make sure to extract band1 and band2 at 250m and band 7 at 5000m.

import os
from osgeo import gdal

# Define directories
hdf_directory = "path/to/data/download/MODIS/"  # Directory with .hdf files
output_base_directory = "path/to/data/download/MODIS/extracted_bands"  # Base directory to save extracted bands
os.makedirs(output_base_directory, exist_ok=True)

# Define band indices for MOD09GA and MOD09GQ products
bands_to_extract_mod09ga = {
    "sur_refl_b07": 18  # Only extract band 7 for MOD09GA
}

bands_to_extract_mod09gq = {
    "sur_refl_b01": 0,  # Extract bands 1 and 2 for MOD09GQ
    "sur_refl_b02": 1
}

# Function to extract bands from HDF file using GDAL
def extract_bands_from_hdf(hdf_file, bands_to_extract, scene_id):
    try:
        # Open the HDF file using GDAL
        hdf_dataset = gdal.Open(hdf_file)

        if hdf_dataset is None:
            print(f"Failed to open {hdf_file}")
            return

        # Get subdatasets (GDAL lists all the datasets within the HDF file)
        subdatasets = hdf_dataset.GetSubDatasets()

        # Log subdatasets for debugging
        print(f"Subdatasets found in {hdf_file}:")
        for i, subdataset in enumerate(subdatasets):
            print(f"{i}: {subdataset[0]}")

        # Create scene output directory based on the last 13 digits of the filename
        scene_output_directory = os.path.join(output_base_directory, scene_id)
        os.makedirs(scene_output_directory, exist_ok=True)

        for band_name, index in bands_to_extract.items():
            # Get the subdataset path for the required band
            if index < len(subdatasets):
                subdataset_path = subdatasets[index][0]

                # Open the subdataset
                band_dataset = gdal.Open(subdataset_path)

                if band_dataset is None:
                    print(f"Failed to open subdataset {band_name} in {hdf_file}")
                    continue

                # Construct the output filename
                output_filename = f"{scene_id}_{band_name}.tif"
                output_path = os.path.join(scene_output_directory, output_filename)

                # Save the band as a GeoTIFF
                gdal.Translate(output_path, band_dataset, format='GTiff')
                print(f"Saved {band_name} to {output_path}")
            else:
                print(f"Index {index} out of range for subdatasets in {hdf_file}")

    except Exception as e:
        print(f"Error processing {hdf_file}: {e}")

# Loop over all HDF files in the directory
for hdf_file in os.listdir(hdf_directory):
    if hdf_file.endswith(".hdf"):
        hdf_file_path = os.path.join(hdf_directory, hdf_file)

        # Extract the last 13 digits for the scene folder name
        scene_id = hdf_file[-17:-4]  # Last 13 digits before ".hdf"

        print(f"Processing file: {hdf_file_path}")

        # Check if the file starts with MOD09GA or MOD09GQ
        if hdf_file.startswith("MOD09GA"):
            # Extract only sur_refl_b07 for MOD09GA
            extract_bands_from_hdf(hdf_file_path, bands_to_extract_mod09ga, scene_id)
        elif hdf_file.startswith("MOD09GQ"):
            # Extract sur_refl_b01 and sur_refl_b02 for MOD09GQ
            extract_bands_from_hdf(hdf_file_path, bands_to_extract_mod09gq, scene_id)

print("Processing complete.")


## Downloading Sentinel 1 GRD data 

In [ ]:
#Example of Single shapefile

from os import listdir
import getpass
import geopandas as gpd
from tqdm import tqdm
import asf_search as asf
from datetime import datetime

# User authentication
username = input('Username:')
password = getpass.getpass('Password:')

# Start ASF session with credentials
session = asf.ASFSession().auth_with_creds(username=username, password=password)

# Load the shapefile as AOI
shapefile_path = 'path/to/shapefiles/j3zo_extent.shp'  # Update this with the path to your shapefile
gdf = gpd.read_file(shapefile_path)
aoi_geom = gdf.geometry.unary_union.wkt  # Combine all geometries into one and get WKT

# Define date range
start_date = '2022-10-01'  # Update with your start date
end_date = '2022-11-28'    # Update with your end date

# Search for Sentinel-1 GRD HD data with the specified date range
results = asf.geo_search(
    intersectsWith=aoi_geom,
    platform=asf.PLATFORM.SENTINEL1,
    processingLevel=asf.PRODUCT_TYPE.GRD_HD,
    start=datetime.strptime(start_date, '%Y-%m-%d'),
    end=datetime.strptime(end_date, '%Y-%m-%d'),
    maxResults=2
)

# Download with progress bar using tqdm and parallel processes
for result in tqdm(results, desc="Downloading GRD HD Images", unit="file"):
    result.download(
        path='path/to/output/GRD',
        session=session,
        #processes=2  # Use 2 parallel download processes
    )

# List the downloaded files
print(listdir('path/to/output/GRD'))


## Dwonloading Sen1 data using multiple shapefile

In [ ]:
##Iterating over each shapefile and saving in the corresponding folder

from os import listdir, makedirs
import os
import getpass
import geopandas as gpd
from tqdm import tqdm
import asf_search as asf
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

# User authentication
username = input('Username:')
password = getpass.getpass('Password:')

# Start ASF session with credentials
session = asf.ASFSession().auth_with_creds(username=username, password=password)

# Define the folder where shapefiles are located
shapefile_folder = 'path/'  # Update this with the folder containing your shapefiles

# Define date range
start_date = '2022-10-01'  # Update with your start date
end_date = '2022-11-30'    # Update with your end date

# Function to download a single product and save in specific folder
def download_product(product, download_path):
    product.download(path=download_path, session=session)

# Function to process each shapefile and download Sentinel-1 data
def process_shapefile(shapefile_path):
    # Load the shapefile as AOI
    gdf = gpd.read_file(shapefile_path)
    aoi_geom = gdf.geometry.union_all().wkt  # Updated for handling the geometry

    # Search for Sentinel-1 GRD HD data with the specified date range
    results = asf.geo_search(
        intersectsWith=aoi_geom,
        platform=asf.PLATFORM.SENTINEL1,
        processingLevel=asf.PRODUCT_TYPE.GRD_HD,
        start=datetime.strptime(start_date, '%Y-%m-%d'),
        end=datetime.strptime(end_date, '%Y-%m-%d'),
        maxResults=20
    )

    # Create a folder named after the shapefile to save the images
    shapefile_name = os.path.splitext(os.path.basename(shapefile_path))[0]
    download_folder = f'path/to/data/download/GRD/{shapefile_name}'  # Folder named after shapefile
    if not os.path.exists(download_folder):
        makedirs(download_folder)  # Create the folder if it doesn't exist

    # Download with progress bar using tqdm and parallel processes
    with ThreadPoolExecutor(max_workers=2) as executor:  # Use 2 parallel threads
        list(tqdm(executor.map(lambda product: download_product(product, download_folder), results), 
                  total=len(results), 
                  desc=f"Downloading GRD HD Images for {shapefile_name}", 
                  unit="file"))

# Iterate through all shapefiles in the folder and process them
for shapefile in listdir(shapefile_folder):
    if shapefile.endswith('.shp'):
        shapefile_path = os.path.join(shapefile_folder, shapefile)
        print(f"Processing shapefile: {shapefile}")
        process_shapefile(shapefile_path)

# List the downloaded files
for shapefile in listdir(shapefile_folder):
    if shapefile.endswith('.shp'):
        shapefile_name = os.path.splitext(shapefile)[0]
        download_folder = f'path/to/data/download/GRD/{shapefile_name}'
        print(f"Files in {download_folder}: {listdir(download_folder)}")
